In [1]:
"""
Quantum Teleportation — Pure Python reference implementation (no external deps)
------------------------------------------------------------------------------
This module implements the standard 3‑qubit quantum teleportation protocol **without**
Qiskit or NumPy, so it runs in restricted environments. It uses small, explicit
state‑vectors (length 8) and applies gates by manipulating amplitudes directly.

Qubit order (big‑endian index): [q0, q1, q2]
- q0: Alice's input |ψ⟩ to be teleported
- q1: Alice's half of the EPR pair
- q2: Bob's half of the EPR pair (receives |ψ⟩ after classical communication)

Index mapping for basis |q0 q1 q2⟩ → integer: idx = (q0<<2) | (q1<<1) | q2

Protocol steps implemented:
1) Prepare |ψ⟩ on q0, and |00⟩ on q1q2.
2) Create Bell pair between q1 and q2 with H(q1); CNOT(q1→q2).
3) Alice performs Bell‑basis measurement on q0q1 via CNOT(q0→q1); H(q0).
4) Measure q0 and q1 → classical bits m0 (from q0) and m1 (from q1).
5) Bob applies corrections on q2: if m1==1 apply X; if m0==1 apply Z.
6) Bob's qubit equals the original |ψ⟩ (up to global phase).

Utilities included:
- Single‑qubit gates (H, X, Z) on arbitrary qubits.
- Two‑qubit CNOT with arbitrary control/target.
- Projective measurement of one or more qubits, with reproducible RNG.
- Fidelity check versus the original |ψ⟩ and compact tests.

Author: ChatGPT
"""

from __future__ import annotations
from typing import List, Tuple, Dict
import math
import random

# ----------------------------
# Basic linear‑algebra helpers
# ----------------------------
ComplexVec = List[complex]


def zero_state(n: int) -> ComplexVec:
    v = [0j] * (1 << n)
    v[0] = 1 + 0j
    return v


def normalize(v: ComplexVec) -> ComplexVec:
    norm = math.sqrt(sum((abs(x) ** 2) for x in v))
    if norm == 0:
        return v
    return [x / norm for x in v]


def bit(value: int, k: int) -> int:
    """Return bit k (0 or 1) of value, where k=0 is LSB.
    Our global indexing uses big‑endian for (q0,q1,q2), but per‑index bit access
    still uses standard integer bits (q2 at bit 0, q1 at bit 1, q0 at bit 2).
    """
    return (value >> k) & 1


# ----------------------------
# Gate application on 3 qubits
# ----------------------------
# Single‑qubit gates are applied in‑place by transforming pairs of amplitudes that differ
# at the target qubit. We keep qubit indices (0=q0, 1=q1, 2=q2) consistent everywhere.

H = [[1 / math.sqrt(2), 1 / math.sqrt(2)], [1 / math.sqrt(2), -1 / math.sqrt(2)]]
X = [[0, 1], [1, 0]]
Z = [[1, 0], [0, -1]]


def apply_single_qubit_gate(state: ComplexVec, gate: List[List[float]], q: int) -> None:
    n = 3  # fixed in this module
    size = 1 << n
    # In the integer index, q2 is bit 0, q1 is bit 1, q0 is bit 2.
    bitpos = 2 - q  # map q0→2, q1→1, q2→0
    new = state.copy()
    for idx in range(size):
        if bit(idx, bitpos) == 0:
            idx0 = idx
            idx1 = idx | (1 << bitpos)
            a0 = state[idx0]
            a1 = state[idx1]
            # [a0', a1']^T = gate * [a0, a1]^T
            new[idx0] = gate[0][0] * a0 + gate[0][1] * a1
            new[idx1] = gate[1][0] * a0 + gate[1][1] * a1
    state[:] = new


def apply_cnot(state: ComplexVec, control_q: int, target_q: int) -> None:
    if control_q == target_q:
        raise ValueError("control and target must differ")
    n = 3
    size = 1 << n
    cpos = 2 - control_q
    tpos = 2 - target_q
    new = state.copy()
    for idx in range(size):
        if bit(idx, cpos) == 1 and bit(idx, tpos) == 0:
            j = idx | (1 << tpos)  # flip target
            new[idx] = state[j]
            new[j] = state[idx]
    state[:] = new


# ----------------------------
# Measurement
# ----------------------------

def measure_qubit(state: ComplexVec, q: int, rng: random.Random) -> Tuple[int, ComplexVec]:
    n = 3
    size = 1 << n
    pos = 2 - q
    p1 = sum(abs(state[i]) ** 2 for i in range(size) if bit(i, pos) == 1)
    r = rng.random()
    outcome = 1 if r < p1 else 0
    # collapse and renormalize
    new = [0j] * size
    for i in range(size):
        if bit(i, pos) == outcome:
            new[i] = state[i]
    new = normalize(new)
    return outcome, new


# ----------------------------
# State preparation
# ----------------------------

def prepare_psi_on_q0(theta: float, phi: float) -> ComplexVec:
    """Return 3‑qubit state with |ψ⟩ on q0 and |00⟩ on q1q2.
    |ψ⟩ = cos(theta/2)|0⟩ + e^{i phi} sin(theta/2)|1⟩
    """
    a = math.cos(theta / 2.0)
    b = math.sin(theta / 2.0) * complex(math.cos(phi), math.sin(phi))
    # Start |000⟩ then put |ψ⟩ on q0 via rotation in the subspace of q0
    state = zero_state(3)
    # Equivalent to setting amplitudes for indices with q1=q2=0 (bits ..00)
    # idx for |0 0 0⟩ is 0, for |1 0 0⟩ is 4
    state = [0j] * 8
    state[0] = a
    state[4] = b
    return normalize(state)


# ----------------------------
# Teleportation protocol
# ----------------------------

def teleport(theta: float, phi: float, seed: int = 0) -> Tuple[int, int, ComplexVec, ComplexVec, float]:
    """Run teleportation for |ψ⟩(theta, phi).
    Returns (m0, m1, final_state, bob_qubit_vec, fidelity).
    """
    rng = random.Random(seed)

    # 1) Prepare |ψ⟩ on q0, |00⟩ on q1q2
    state = prepare_psi_on_q0(theta, phi)

    # 2) Create Bell pair between q1 and q2
    apply_single_qubit_gate(state, H, q=1)
    apply_cnot(state, control_q=1, target_q=2)

    # 3) Bell‑basis measurement transform on q0,q1
    apply_cnot(state, control_q=0, target_q=1)
    apply_single_qubit_gate(state, H, q=0)

    # 4) Measure q0 and q1 (Alice)
    m0, state = measure_qubit(state, q=0, rng=rng)
    m1, state = measure_qubit(state, q=1, rng=rng)

    # 5) Bob’s corrections on q2
    if m1 == 1:
        apply_single_qubit_gate(state, X, q=2)
    if m0 == 1:
        apply_single_qubit_gate(state, Z, q=2)

    # Extract Bob’s qubit vector (q2) for the observed m0,m1 (q0,q1 are now classical)
    # Build the 2‑vector [amp(|..0⟩), amp(|..1⟩)] where the .. are fixed to m0,m1
    def idx(b2: int, b1: int, b0: int) -> int:
        return (b2 << 2) | (b1 << 1) | b0

    v_bob = [state[idx(m0, m1, 0)], state[idx(m0, m1, 1)]]
    v_bob = normalize(v_bob)

    # Target single‑qubit vector for |ψ⟩
    a = math.cos(theta / 2.0)
    b = math.sin(theta / 2.0) * complex(math.cos(phi), math.sin(phi))
    psi = normalize([a, b])

    # Fidelity |⟨ψ|φ⟩|^2 (global phase invariant)
    inner = (psi[0].conjugate() * v_bob[0] + psi[1].conjugate() * v_bob[1])
    fidelity = abs(inner) ** 2

    return m0, m1, state, v_bob, fidelity


# ----------------------------
# Pretty helpers & tests
# ----------------------------

def pretty(v: ComplexVec, precision: int = 5) -> str:
    def fmt(c: complex) -> str:
        return f"{c.real:.{precision}f}{c.imag:+.{precision}f}j"
    return "[" + ", ".join(fmt(x) for x in v) + "]"


def run_basic_tests() -> None:
    print("Quantum Teleportation — Pure Python tests\n")

    tests = [
        (0.0, 0.0, "|0⟩"),
        (math.pi, 0.0, "|1⟩"),
        (math.pi / 2, 0.0, "|+⟩"),
        (math.pi / 2, math.pi, "|−⟩"),
        (math.pi / 2, math.pi / 2, "|+i⟩"),
        (1.234, 2.345, "random(θ,φ)"),
    ]

    for (theta, phi, name) in tests:
        m0, m1, state, v_bob, F = teleport(theta, phi, seed=42)
        print(f"Test {name:10s}  outcome m0m1={m0}{m1}  Bob={pretty(v_bob,3)}  fidelity={F:.6f}")
        assert F > 1 - 1e-12, f"Fidelity too low: {F} for {name}"

    print("\n✅ All teleportation tests passed (fidelity≈1 for varied states).")


if __name__ == "__main__":
    run_basic_tests()


Quantum Teleportation — Pure Python tests

Test |0⟩         outcome m0m1=01  Bob=[1.000+0.000j, 0.000+0.000j]  fidelity=1.000000
Test |1⟩         outcome m0m1=01  Bob=[0.000+0.000j, 1.000+0.000j]  fidelity=1.000000
Test |+⟩         outcome m0m1=01  Bob=[0.707+0.000j, 0.707+0.000j]  fidelity=1.000000
Test |−⟩         outcome m0m1=01  Bob=[0.707+0.000j, -0.707+0.000j]  fidelity=1.000000
Test |+i⟩        outcome m0m1=01  Bob=[0.707+0.000j, 0.000+0.707j]  fidelity=1.000000
Test random(θ,φ)  outcome m0m1=01  Bob=[0.816+0.000j, -0.405+0.414j]  fidelity=1.000000

✅ All teleportation tests passed (fidelity≈1 for varied states).
